In [0]:
from zipfile import ZipFile
filename = "data1.zip"

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print('Done')

In [0]:
# import the necessary pacakges
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import Model
from keras.layers import Input
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras import optimizers
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import os

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
print(tf.test.is_built_with_cuda())

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from imutils import paths
import os
import cv2

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [0]:
model.compile(loss="binary_crossentropy", 
              optimizer=optimizers.RMSprop(lr=1e-4), 
              metrics=["accuracy"])

In [0]:
## Training using Generators
from keras.preprocessing.image import ImageDataGenerator

train_dir = "data1//train"
val_dir = "data1//val"
test_dir = "data1//test"

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150,150),batch_size=32,class_mode="binary")
validation_generator = val_datagen.flow_from_directory(val_dir,target_size=(150,150),batch_size=32,class_mode="binary")
test_generator = test_datagen.flow_from_directory(val_dir,target_size=(150,150),batch_size=32,class_mode="binary")

In [0]:
train_steps_per_epoch = np.math.ceil(train_generator.samples / train_generator.batch_size)
val_steps_per_epoch = np.math.ceil(validation_generator.samples / validation_generator.batch_size)

result = model.fit_generator(train_generator,
                             steps_per_epoch=train_steps_per_epoch,
                             epochs=40,
                             validation_data=validation_generator,
                             validation_steps=val_steps_per_epoch,
                             workers = 10)
model.save("CatsAndDogs.hdf5")

In [0]:
# Plot the training and validation loss
import matplotlib.pyplot as plt
loss = result.history['loss']
val_loss = result.history['val_loss']
epochs = range(1,len(loss)+1)
plt.plot(epochs,loss,'bo',label='Training loss') # blue dots
plt.plot(epochs,val_loss,'b',label='Validation loss') # blue line
plt.title("Training and validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# plot the training and validation accuracy
plt.clf()
acc_values = result.history['accuracy']
val_acc_values = result.history['val_accuracy']
plt.plot(epochs,acc_values,'bo',label='Training accuracy')
plt.plot(epochs,val_acc_values,'b',label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [0]:
# Test Eval
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
final_test_loss, final_test_acc = model.evaluate_generator(
    test_generator,
    steps=test_steps_per_epoch
)

In [0]:
# Test Plot
print("Test Accuracy : ", final_test_acc * 100)
print("Test Loss : ", final_test_loss)

In [0]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt
datagen = ImageDataGenerator(rotation_range=40,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

train_cats_dir = "data1//train//cats"
fnames = [os.path.join(train_cats_dir,fname) for fname in os.listdir(train_cats_dir)]
img_path = fnames[3]
img = image.load_img(img_path,target_size = (150,150))
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
for batch in datagen.flow(x,batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i +=1
    if i%4 == 0:
        break

plt.show()

In [0]:
from keras.applications import VGG16

conv_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150,150,3)
)
conv_base.summary()

In [0]:
conv_base.trainable = False

modelvgg_froz = models.Sequential()
modelvgg_froz.add(conv_base)
modelvgg_froz.add(layers.Flatten())
modelvgg_froz.add(layers.Dense(512, activation='relu'))
modelvgg_froz.add(layers.Dense(1, activation='sigmoid'))
modelvgg_froz.summary()

modelvgg_froz.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=1e-4),
    metrics=['acc']
)

In [0]:
history_vgg_frozen=modelvgg_froz.fit_generator(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=val_steps_per_epoch
)
modelvgg_froz.save("CatsAndDogs_frozen.hdf5")


In [0]:
import matplotlib.pyplot as plt
hist_dict = history_vgg_frozen.history
print(hist_dict.keys())
loss_values = hist_dict['loss']
val_loss_values = hist_dict['val_loss']
epochs = range(1, len(hist_dict['val_loss']) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training Loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.savefig('Frozen_loss.png')

plt.clf()
acc_values = hist_dict['acc']
val_acc_values = hist_dict['val_acc']

plt.plot(epochs, acc_values, 'bo', label='Training Acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation Acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig('Frozen_acc.png')



In [0]:
# Test Eval
frozen_test_loss, frozen_test_acc = modelvgg_froz.evaluate_generator(
    test_generator,
    steps=test_steps_per_epoch
)

In [0]:
# Test Plot
print("Test Accuracy : ", frozen_test_acc * 100)
print("Test Loss : ", frozen_test_loss)

In [0]:
img_path = 'data1/test/cats/1989.jpg'
from keras.preprocessing import image
import numpy as np
img = image.load_img(img_path, target_size = (150,150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255
#print (img_tensor.shape)

import matplotlib.pyplot as plt
plt.imshow(img_tensor[0])
plt.show()

from keras.models import load_model
model_loaded = load_model('CatsAndDogs.hdf5')
from keras import models
print(model_loaded.layers)
layer_outputs = []
for layer in model_loaded.layers[:8]:
    layer_outputs.append(layer.output)
#layer_outputs = [layer.ouput for layer in model_loaded.layers[:8]]

activation_model = models.Model(inputs=model_loaded.input, outputs= layer_outputs)
activations = activation_model.predict(img_tensor)

first_layer_activation = activations[0]
#print (first_layer_activation.shape)

import matplotlib.pyplot as plt
plt.matshow(first_layer_activation[0,:,:,4], cmap = 'viridis' )
plt.matshow(first_layer_activation[0,:,:,7], cmap = 'viridis' )

layer_names = []
for layer in model_loaded.layers[:8]:
    layer_names.append(layer.name)
print (layer_names)    


In [0]:
from keras import backend as K
model=load_model('CatsAndDogs.hdf5')  

def deprocess_image(x):
    x-=x.mean()
    x/=(x.std()+1e-5)
    x*=0.1
    x+=0.5
    x=np.clip(x,0,1)
    x*=255
    #Clip to [0,255] and convert to unsigned byte channels
    x=np.clip(x,0,255).astype('uint8')
    return x

def generate_pattern(layer_name,filter_index,size=150):
    #Define Output and Loss
    layer_output=model.get_layer(layer_name).output
    loss=K.mean(layer_output[:,:,:,filter_index])
    #Compute the gradient of the input with respect to loss
    grads=K.gradients(loss,model.input)[0]
    grads/=((K.sqrt(K.mean(K.square(grads))))+1e-5)
    iterate=K.function([model.input], [loss,grads])
    input_img_data=np.random.random((1,size,size,3))*20+128
   
    step=1
    for i in range(100):
        loss_value,grads_value=iterate([input_img_data])
        input_img_data+=grads_value*step
       
    img=input_img_data[0]
    return deprocess_image(img)


layer_names=[]
conv_ly1=model.layers[0].name
layer_names.append(conv_ly1)
conv_ly2=model.layers[2].name
layer_names.append(conv_ly2)
conv_ly3=model.layers[4].name
layer_names.append(conv_ly3)


#layer_outputs=[layer.output for layer in model.layers[:8]]

k=0
#for layer_name,layer in zip(layer_names,model.layers[i]):
for layer_name in layer_names:
    layer=model.layers[k]
    layer_output=layer.output
    print(layer_output.shape)
    size=int(layer_output.shape[1])
    row=8
    col=int(layer_output.shape[3]//row)
    results=np.zeros((row*size,col*size,3)).astype('uint0')
   
    for i in range(row):
        for j in range(col):
            filter_img=generate_pattern(layer_name,i+(j*8),size=size)        
            horizontal_start=i*size
            horizontal_end=horizontal_start+size
            vertical_start=j*size
            vertical_end=vertical_start+size
            results[horizontal_start:horizontal_end,vertical_start:vertical_end,:]=filter_img[:,:,:]
    plt.figure(figsize=(20,20))
    plt.title(layer_name)
    plt.imshow(results)
    k+=2

In [0]:
conv_base.trainable = True

modelvgg_froz.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=1e-4),
    metrics=['acc']
)

history_vgg_unfrozen=modelvgg_froz.fit_generator(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=val_steps_per_epoch
)
modelvgg_froz.save('CatsAndDogs_unfrozen.hdf5')

In [0]:
import matplotlib.pyplot as plt
hist_dict = history_vgg_unfrozen.history
print(hist_dict.keys())
loss_values = hist_dict['loss']
val_loss_values = hist_dict['val_loss']
epochs = range(1, len(hist_dict['val_loss']) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training Loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.savefig('Unfrozen_loss.png')

plt.clf()
acc_values = hist_dict['acc']
val_acc_values = hist_dict['val_acc']

plt.plot(epochs, acc_values, 'bo', label='Training Acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation Acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig('Unfrozen_acc.png')

In [0]:
# Test Eval
unfroz_test_loss, unfroz_test_acc = modelvgg_froz.evaluate_generator(
    test_generator,
    steps=test_steps_per_epoch
)

In [0]:
# Test Plot
print("Test Accuracy : ", unfroz_test_acc)
print("Test Loss : ", unfroz_test_loss)

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

# val_gen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
#     )

# test_gen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
#     )
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(150,150),batch_size=32,class_mode="binary")


# val_generator = train_gen.flow_from_directory(
#     '/content/DL4_data/validation',
#     target_size=(150, 150),
#     batch_size=50,
#     class_mode='binary'
# )

# test_generator = train_gen.flow_from_directory(
#     '/content/DL4_data/test',
#     target_size=(150, 150),
#     batch_size=50,
#     class_mode='binary'
# )

In [0]:
# conv_base = VGG16(
#     weights='imagenet',
#     include_top=False,
#     input_shape=(150,150,3)
# )

conv_base.trainable = False

# modelvgg_froz_da = models.Sequential()
# modelvgg_froz_da.add(conv_base)
# modelvgg_froz_da.add(layers.Flatten())
# modelvgg_froz_da.add(layers.Dropout(0.3))
# modelvgg_froz_da.add(layers.Dense(512, activation='relu'))
# modelvgg_froz_da.add(layers.Dense(1, activation='sigmoid'))
# modelvgg_froz_da.summary()

modelvgg_froz.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=1e-4),
    metrics=['acc']
)
history_vgg_frozen_da=modelvgg_froz.fit_generator(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=val_steps_per_epoch
)


In [0]:
modelvgg_froz.save('cats_and_dogs-frozen-da.hdf5')

In [0]:
import matplotlib.pyplot as plt
hist_dict = history_vgg_frozen_da.history
print(hist_dict.keys())
loss_values = hist_dict['loss']
val_loss_values = hist_dict['val_loss']
epochs = range(1, len(hist_dict['val_loss']) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training Loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.savefig('Frozenda_loss.png')

plt.clf()
acc_values = hist_dict['acc']
val_acc_values = hist_dict['val_acc']

plt.plot(epochs, acc_values, 'bo', label='Training Acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation Acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig('Frozenda_acc.png')

In [0]:
da_test_loss, da_test_acc = modelvgg_froz.evaluate_generator(
    test_generator,
    steps=test_steps_per_epoch
)

In [0]:
print("Test Accuracy : ", da_test_acc)
print("Test Loss : ", da_test_loss)